In [21]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm

%matplotlib inline

In [29]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from sklearn.model_selection import train_test_split

In [22]:
# load the files
houselist = glob.glob('HousesDataset/*.jpg')

image_data = []
for fname in houselist:
    image_name = fname.split('/')[1].split('.')[0]
    image_data.append((image_name, Image.open(fname)))
    pass

text_data = pd.read_csv('HousesDataset/HousesInfo.txt', sep=' ',
                        names=["nBeds", "nbaths", "area", "zipcode", "price"])

In [23]:
image_same_size = []
for name, image in tqdm(image_data):
    image_same_size.append((name, image.resize((300, 300))))

100%|█████████████████████████████████████████████████████████████████████████████| 2140/2140 [00:17<00:00, 123.29it/s]


In [24]:
X = np.zeros((535, 300, 300, 12)).astype(int)
for name, image in tqdm(image_data):
    index, room = name.split('_')
    if room == "bathroom":
        X[int(index) - 1, :, :, 0:3] = np.array(image.resize((300, 300)))
    elif room == "bedroom":
        X[int(index) - 1, :, :, 3:6] = np.array(image.resize((300, 300)))
    elif room == "frontal":
        X[int(index) - 1, :, :, 6:9] = np.array(image.resize((300, 300)))
    else:
        X[int(index) - 1, :, :, 9:12] = np.array(image.resize((300, 300)))

100%|█████████████████████████████████████████████████████████████████████████████| 2140/2140 [00:02<00:00, 852.73it/s]


In [37]:
text_array = np.zeros((535, 5))
for i, text in text_data.iterrows():
    text_array[i] = text.values

In [46]:
y = text_array[:, -1]
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(300, 300, 12)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adadelta())

model.fit(x_train, y_train,
          batch_size=32,
          epochs=100,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Instructions for updating:
Use tf.cast instead.
Train on 358 samples, validate on 177 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[1401856,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_1/random_uniform/RandomUniform (defined at C:\Users\Zhenhua Wang\Anaconda\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4139) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_1/random_uniform/RandomUniform', defined at:
  File "C:\Users\Zhenhua Wang\Anaconda\envs\tf_gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Zhenhua Wang\Anaconda\envs\tf_gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Zhenhua Wang\Anaconda\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Zhenhua Wang\Anaconda\envs\tf_gpu\lib\site-packages\t

In [38]:
text_array

array([[4.0000e+00, 4.0000e+00, 4.0530e+03, 8.5255e+04, 8.6950e+05],
       [4.0000e+00, 3.0000e+00, 3.3430e+03, 3.6372e+04, 8.6520e+05],
       [3.0000e+00, 4.0000e+00, 3.9230e+03, 8.5266e+04, 8.8900e+05],
       ...,
       [3.0000e+00, 2.0000e+00, 2.0140e+03, 9.4531e+04, 4.0700e+05],
       [4.0000e+00, 3.0000e+00, 2.3120e+03, 9.4531e+04, 4.1900e+05],
       [5.0000e+00, 3.0000e+00, 3.7960e+03, 9.4531e+04, 6.1500e+05]])